# Progetto d'esame di Data Analysis in Experimental Physics with Machine Learning

Gruppo composto dagli studenti Luca Attinà, Sharis Feriotto e Matteo Marchisio Caprioglio

Dataset ipotesi: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset

In [1]:
import matplotlib.pyplot as plt

In [2]:
%conda install -c conda-forge kagglehub


Jupyter detected...
Note: you may need to restart the kernel to use updated packages.

Channels:
 - conda-forge
 - defaults
Platform: win-64
Solving environment: done

# All requested packages already installed.



In [3]:
!pip install matplotlib

In [4]:
# Prova import dataset
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vipoooool/new-plant-diseases-dataset")

print("Path to dataset files:", path)


C:\Users\Itsashashi\miniconda3\envs\tf-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\Itsashashi\.cache\kagglehub\datasets\vipoooool\new-plant-diseases-dataset\versions\2


In [5]:
train_path = path + "/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"
val_path = path + "/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid"
test_path = path + "/test"

In [6]:
train_path

'C:\\Users\\Itsashashi\\.cache\\kagglehub\\datasets\\vipoooool\\new-plant-diseases-dataset\\versions\\2/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'

In [7]:
%ls "{train_path}"
%ls "{val_path}"

 Volume in drive C is OS
 Volume Serial Number is 90BA-7656

 Directory of C:\Users\Itsashashi\.cache\kagglehub\datasets\vipoooool\new-plant-diseases-dataset\versions\2\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)\train

09/07/2025  20:57    <DIR>          .
09/07/2025  20:57    <DIR>          ..
09/07/2025  20:56    <DIR>          Apple___Apple_scab
09/07/2025  20:56    <DIR>          Apple___Black_rot
09/07/2025  20:56    <DIR>          Apple___Cedar_apple_rust
09/07/2025  20:56    <DIR>          Apple___healthy
09/07/2025  20:56    <DIR>          Blueberry___healthy
09/07/2025  20:56    <DIR>          Cherry_(including_sour)___healthy
09/07/2025  20:56    <DIR>          Cherry_(including_sour)___Powdery_mildew
09/07/2025  20:56    <DIR>          Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
09/07/2025  20:56    <DIR>          Corn_(maize)___Common_rust_
09/07/2025  20:56    <DIR>          Corn_(maize)___healthy
09/07/2025  20:56    <DIR>          

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models

In [26]:
img_size = (256, 256)
batch_size = 64

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_path,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical',
    color_mode= "rgb"
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_path,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical',
    color_mode= "rgb"

)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_path,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical',
    color_mode= "rgb"

)

test_ds

Found 70295 files belonging to 38 classes.
Found 17572 files belonging to 38 classes.
Found 33 files belonging to 1 classes.


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

In [27]:
# ipotesi modello
# convolutional 2d
# relu activation
# maxpooling forse?


In [28]:
# Get number of classes from dataset
num_classes = len(train_ds.class_names)


In [29]:
# Define the model
model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(256, 256, 3)),
    
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),  # helps prevent overfitting
    layers.Dense(num_classes, activation='softmax')
])

In [30]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

In [31]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=4,
    callbacks=[early_stop]
)


Epoch 1/4
 255/1099 ━━━━━━━━━━━━━━━━━━━━ 42:48 3s/step - accuracy: 0.0665 - loss: 3.5521

In [ ]:
# Evaluate on test set
test_loss, test_acc = model.evaluate(test_ds)
print(f"\n✅ Test Accuracy: {test_acc:.4f}")

In [ ]:

# Plot accuracy
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()